In [ ]:
'''
Final Assignment: Data driven approach to laminar and turbulent flow classification

Laminar flow has been characterized for its smooth and linear behavior concerning its flow.
When a non-linearity is introduced, the flow becomes turbulent.
Visually it is possible to distinguish between laminar and turbulent flow by
identifying areas in the flow that become chaotic or non-smooth.
We used a Convolutional Neural Network (CNN) trained on labeled laminar and
turbulent images to automate the process. In this assignment, we take simulated
contours from a flow past cylinder simulation, train it, and test it to see how
effective the CNN is at distinguishing between the laminar and turbulent flow.
The geometry is as follows -
There is a circular cylinder axis along the z-axis(out of the plane),
you have to simulate 2d velocity profile about it.

Objective of CNN model is that given a simulated velocity profile,
it should be able to efficiently predict the nature of fluid flow.
Compare the accuracy of different CNN architectures covered in the class and
based on that choose a model of your dataset.
Then optimize hyperparameters to get accuracy more than 80%

 Deadline to the assignment is : 10th May
'''

In [ ]:
'''
1. create  and preprocess
- 10 laminar/10 turbulent images for dataset
- preprocess - resize, normalise
2. try some cnns
3. optimise hyperparameters (80% accuracy)
4. documentation
'''

In [ ]:
'''
3. Model Training and Evaluation:
   - Train the selected CNN model on the training set.
   - Validate the model on the testing set to evaluate its performance.
   - Fine-tune the model to improve its accuracy.

4. Hyperparameter Optimization:
   - Tune hyperparameters such as learning rate, batch size, number of epochs, etc.
   - Use techniques like grid search or random search to find optimal hyperparameters.
   - Aim to achieve an accuracy of more than 80% as per the assignment requirements.

6. Report and Documentation:
   - Document your methodology, results, and observations.
'''

In [17]:
# importing libraries

import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical

In [18]:
# setting a seed to ensure reproducibility

np.random.seed(42)
tf.random.set_seed(42)

In [19]:
# GPU

print("GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

laminar_dir = "/content/drive/MyDrive/Fluid_flow_data/laminar"
laminar_dir_p = "/content/drive/MyDrive/Fluid_flow_data/laminarp"

turbulant_dir = "/content/drive/MyDrive/Fluid_flow_data/turbulent"
turbulant_dir_p = "/content/drive/MyDrive/Fluid_flow_data/turbulantp"

#source_dir = "/content/drive/MyDrive/Fluid_flow_data"
train_dir = "/content/drive/MyDrive/Fluid_flow_data/train"
test_dir = "/content/drive/MyDrive/Fluid_flow_data/test"
val_dir = "/content/drive/MyDrive/Fluid_flow_data/validation"

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(laminar_dir_p, exist_ok=True)
os.makedirs(turbulant_dir_p, exist_ok=True)

lamtrain = os.path.join(train_dir, "laminar")
lamtest = os.path.join(test_dir, "laminar")
lamval = os.path.join(val_dir, "laminar")

turtrain = os.path.join(train_dir, "turbulant")
turtest = os.path.join(test_dir, "turbulant")
turval = os.path.join(val_dir, "turbulant")

os.makedirs(lamtrain, exist_ok=True)
os.makedirs(lamtest, exist_ok=True)
os.makedirs(lamval, exist_ok=True)
os.makedirs(turtrain, exist_ok=True)
os.makedirs(turtest, exist_ok=True)
os.makedirs(turval, exist_ok=True)

#train_dir = r"/content/drive/MyDrive/Fluid_flow_data"


GPUs Available:  0


In [22]:
img_width, img_height = 150, 150
num_classes = 2
batchsize = 10

In [23]:
# function to preprocess images
def preprocess_images(source_dir, target_dir):
      for file in os.listdir(source_dir):
          # load image
          img = load_img(os.path.join(source_dir, file), target_size=(img_width, img_height))
          img_array = img_to_array(img)
          img_array /= 255.0  # normalize pixel values
          # move preprocessed image to target directory
          # os.rename(os.path.join(subdir, file), os.path.join(target_dir, subdir.split('/')[-1] + '_' + file))
          os.rename(os.path.join(source_dir, file), os.path.join(target_dir, file))

preprocess_images(laminar_dir, laminar_dir_p)
preprocess_images(turbulant_dir, turbulant_dir_p)

In [24]:
# split data into train, test, and validation
laminar_files = os.listdir(laminar_dir_p)
turbulent_files = os.listdir(turbulant_dir_p)

laminar_train, laminar_test_val = train_test_split(laminar_files, test_size=0.3, random_state=42)
laminar_test, laminar_val = train_test_split(laminar_test_val, test_size=0.5, random_state=42)

turbulent_train, turbulent_test_val = train_test_split(turbulent_files, test_size=0.3, random_state=42)
turbulent_test, turbulent_val = train_test_split(turbulent_test_val, test_size=0.5, random_state=42)


In [27]:
def move_files(source_files, source_dir, dest_dir):
    for file in source_files:
        #os.rename(os.path.join(source_dir, file), os.path.join(dest_dir, file))
        os.rename(os.path.join(source_dir, file), os.path.join(dest_dir, file))

In [28]:
# move files to appropriate directories

move_files(laminar_train, laminar_dir_p, os.path.join(train_dir, "laminar"))
move_files(laminar_test, laminar_dir_p, os.path.join(test_dir, "laminar"))
move_files(laminar_val, laminar_dir_p, os.path.join(val_dir, "laminar"))

move_files(turbulent_train, turbulant_dir_p, os.path.join(train_dir, "turbulant"))
move_files(turbulent_test, turbulant_dir_p, os.path.join(test_dir, "turbulant"))
move_files(turbulent_val, turbulant_dir_p, os.path.join(val_dir, "turbulant"))


In [ ]:
'''
lamtrain = os.path.join(train_dir, "laminar")
lamtest = os.path.join(test_dir, "laminar")
lamval = os.path.join(val_dir, "laminar")

turtrain = os.path.join(train_dir, "turbulant")
turtest = os.path.join(test_dir, "turbulant")
turval = os.path.join(val_dir, "turbulant")
'''

In [29]:
# one-hot encoding
# 0 - laminar
# 1 - turbulent

# train
train_files = []
train_files.extend([os.path.join(lamtrain, file) for file in laminar_train])
train_files.extend([os.path.join(turtrain, file) for file in turbulent_train])

train_labels = []
train_labels.extend([0] * len(laminar_train))
train_labels.extend([1] * len(turbulent_train))

train_labels = to_categorical(train_labels, 2)

# test
test_files = []
test_files.extend([os.path.join(lamtest, file) for file in laminar_test])
test_files.extend([os.path.join(turtest, file) for file in turbulent_test])

test_labels = []
test_labels.extend([0] * len(laminar_test))
test_labels.extend([1] * len(turbulent_test))

test_labels = to_categorical(test_labels, 2)

# validation
val_files = []
val_files.extend([os.path.join(lamval, file) for file in laminar_val])
val_files.extend([os.path.join(turval, file) for file in turbulent_val])

val_labels = []
val_labels.extend([0] * len(laminar_val))
val_labels.extend([1] * len(turbulent_val))

val_labels = to_categorical(val_labels,2)


In [45]:
# building the model

model1 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

# compiling the model
model1.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [49]:
# Load and preprocess images
train_images = []

for file in train_files:

    img = load_img(file, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    train_images.append(img_array)

# Convert lists to numpy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)

val_images = []

for file in val_files:

    img = load_img(file, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    val_images.append(img_array)

# Convert lists to numpy arrays
val_images = np.array(val_images)
val_labels = np.array(val_labels)


In [52]:
import tensorflow.keras.backend as K
K.clear_session()


In [53]:
# fitting the model
history = model1.fit(train_images, train_labels, batch_size = 15, validation_data=(val_images, val_labels), epochs=5)


Epoch 1/5
14/14 [==============================] - 15s 1s/step - loss: 0.2340 - accuracy: 0.8911 - val_loss: 0.0673 - val_accuracy: 0.9778
Epoch 2/5
14/14 [==============================] - 15s 1s/step - loss: 0.0985 - accuracy: 0.9752 - val_loss: 0.0192 - val_accuracy: 1.0000
Epoch 3/5
14/14 [==============================] - 23s 2s/step - loss: 0.1091 - accuracy: 0.9752 - val_loss: 0.0503 - val_accuracy: 0.9778
Epoch 4/5
14/14 [==============================] - 20s 1s/step - loss: 0.1103 - accuracy: 0.9802 - val_loss: 0.2288 - val_accuracy: 0.9333
Epoch 5/5
14/14 [==============================] - 22s 2s/step - loss: 0.0973 - accuracy: 0.9604 - val_loss: 0.0893 - val_accuracy: 0.9778


In [59]:
test_images = []

for file in test_files:

    img = load_img(file, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    test_images.append(img_array)

# Convert lists to numpy arrays
test_images = np.array(test_images)
test_labels = np.array(test_labels)


In [61]:
# Evaluating the model
test_loss, test_accuracy = model1.evaluate(test_images, test_labels, batch_size=10)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy*100)

# Predicting classes
predictions = model1.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Generating true classes
true_classes = np.argmax(test_labels, axis=1)

# Generating class labels
class_labels = ["laminar", "turbulent"]

# Generating classification report
print(classification_report(true_classes, predicted_classes, target_names=class_labels))


5/5 [==============================] - 1s 235ms/step - loss: 0.0934 - accuracy: 0.9545
Test Loss: 0.0933522954583168
Test Accuracy: 95.45454382896423
2/2 [==============================] - 1s 239ms/step
              precision    recall  f1-score   support

     laminar       1.00      0.90      0.95        21
   turbulent       0.92      1.00      0.96        23

    accuracy                           0.95        44
   macro avg       0.96      0.95      0.95        44
weighted avg       0.96      0.95      0.95        44



In [62]:
model2 = Sequential([
        Conv2D(96, (11,11), activation='relu', input_shape=(img_width, img_height, 3)),
        MaxPooling2D(3,3),
        Conv2D(256, (5,5), padding='same', activation='relu'),
        Conv2D(256, (3,3), padding='same', activation='relu'),
        MaxPooling2D(3,3),
        Conv2D(384, (3,3), padding='same', activation='relu'),
        Conv2D(384, (3,3), padding='same', activation='relu'),
        MaxPooling2D(3,3),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(2, activation='softmax')
    ])

# compiling the model
model2.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

# print(model.summary())


In [63]:
# fitting the model
history = model2.fit(train_images, train_labels, batch_size = 10, validation_data=(val_images, val_labels), epochs=4)


Epoch 1/4
21/21 [==============================] - 139s 6s/step - loss: 57.5256 - accuracy: 0.4950 - val_loss: 0.6932 - val_accuracy: 0.4667
Epoch 2/4
21/21 [==============================] - 124s 6s/step - loss: 0.6966 - accuracy: 0.4406 - val_loss: 0.6922 - val_accuracy: 0.5333
Epoch 3/4
21/21 [==============================] - 128s 6s/step - loss: 0.7055 - accuracy: 0.4851 - val_loss: 0.6864 - val_accuracy: 0.6222
Epoch 4/4
21/21 [==============================] - 127s 6s/step - loss: 0.6851 - accuracy: 0.5792 - val_loss: 0.5060 - val_accuracy: 0.8889


In [64]:
# Evaluating the model
test_loss, test_accuracy = model2.evaluate(test_images, test_labels, batch_size=10)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy*100)

# Predicting classes
predictions = model2.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Generating true classes
true_classes = np.argmax(test_labels, axis=1)

# Generating class labels
class_labels = ["laminar", "turbulent"]

# Generating classification report
print(classification_report(true_classes, predicted_classes, target_names=class_labels))


5/5 [==============================] - 8s 2s/step - loss: 0.5613 - accuracy: 0.7955
Test Loss: 0.5612590312957764
Test Accuracy: 79.54545617103577
2/2 [==============================] - 5s 1s/step
              precision    recall  f1-score   support

     laminar       0.70      1.00      0.82        21
   turbulent       1.00      0.61      0.76        23

    accuracy                           0.80        44
   macro avg       0.85      0.80      0.79        44
weighted avg       0.86      0.80      0.79        44

